# Filtering fastqs to remove reads mapping to rRNA genes

One of the Bioinformatics refereees major concerns was the possible presence of rRNA expression remaining in our ribo-minus samples. This expression could be considerably variable and distorting our distribution analysis. To address this, we're going to re-run the analysis after removing reads that map to rRNA genes.

So here I'm going to load the current [ensembl plants v40 Arabidopsis annotation](ftp://ftp.ensemblgenomes.org/pub/plants/release-40/gff3/arabidopsis_thaliana]), filter it for rRNA genes, and then make a function that parses a bam file for reads that map in these regions, and filters them out, outputting a new bam files.

This is just a notebook to test the subroutines. It'll then be run individually on each bam file and the matching fastqs.

In [132]:
import os, sys, re, numpy, scipy, json, pysam, matplotlib, time
%matplotlib inline
from matplotlib import pyplot as plt
from parsing_routines.gff_gtf_tools import annotation

In [133]:
annotpath = "pathto/ensembl/release_40/Arabidopsis_thaliana.TAIR10.40.gff3"
annot = annotation(annotpath, filetype="gff3")

In [134]:
annot.set_feature("rRNAs")
rRNA_regions = numpy.unique(numpy.transpose(numpy.vstack((annot.get_selection_chrids(),
                                                          annot.get_selection_starts(),
                                                          annot.get_selection_stops()))),
                            axis=0)

OK, so not lets write some code to filter 

In [135]:
import time
def filter_bam(bamfilename, outfilename, regions_to_filter_out, LOG_EVERY_Nregions=100, LOG_EVERY_Nreads=10, debug=False):
    
    """Takes a bam file and a list of features. Filters out reads mapping to the positions of these features"""
        
    print("Finding reads to filter...")
    reads_to_filter_out=[]
    thisbam = pysam.AlignmentFile(bamfilename, "rb")
    tot_read_pairs = sum([int(x[3]) for x in thisbam.get_index_statistics()])/2
    
    nlogs=1
    counter=0
    for region in regions_to_filter_out:
        readcounter=0
        for read in thisbam.fetch(region[0], int(region[1]), int(region[2])):
            reads_to_filter_out.append(read.query_name)
            readcounter+=1
        if readcounter>0 and debug:
            print("\t{}:{}-{} : {}".format(region[0], region[1], region[2], readcounter))
        
        counter+=1
        if (counter % LOG_EVERY_Nregions)==0:
            print("\tprocessed {these} regions...".format(these=(nlogs*LOG_EVERY_Nregions)))
            nlogs+=1
    
    reads_to_filter_out=set(reads_to_filter_out)
    print("\tfiltering {} read-pairs ({:.2%})...".format(len(reads_to_filter_out),
                                                         len(reads_to_filter_out)/float(tot_read_pairs)))
    
    print("Filtering reads...")
    ngoodreads=0
    nlogs=1
    counter=0
    outfile = pysam.AlignmentFile(outfilename, "wb", template=thisbam)
    t0=time.time()
    for read in thisbam.fetch():
        print(read)
        
        if read.query_name not in reads_to_filter_out:
            outfile.write(read)
            ngoodreads+=1
        else:
            reads_to_filter_out.remove(read.query_name)
        
        print("\t tested in {:.8f}s".format(time.time()-t0))
        
        counter+=1
        if (counter % LOG_EVERY_Nreads)==0:
            print("\tprocessed {} reads ({:.4f}s)...".format((nlogs*LOG_EVERY_Nreads),time.time()-t0))
            nlogs+=1
            break
    
    thisbam.close()
    outfile.close()
    print("Finished.")
    return(ngoodreads, reads_to_filter_out)

In [136]:
# test
bamfilename = "pathto/Col-1/Aligned.out.sorted.bam"
outfilename = "pathto/Col-1/Aligned.out.sorted.rRNAfiltered.bam"
nkept, filtered_out = filter_bam(bamfilename, outfilename, rRNA_regions, debug=True)

Finding reads to filter...
	1:16431026-16431144 : 2
	1:16431511-16431628 : 4
	1:16431990-16432108 : 2
	1:16432471-16432588 : 4
	2:3706-5513 : 6152576
	2:5782-5945 : 50576
	2:5787-5942 : 50459
	3:13532717-13532835 : 2
	3:13593561-13593679 : 2
	3:13594072-13594190 : 4
	3:13594583-13594701 : 4
	3:13595093-13595211 : 1
	3:13596582-13596700 : 1
	3:13597093-13597211 : 1
	3:13597603-13597721 : 2
	3:13598113-13598232 : 3
	3:13598625-13598743 : 10
	3:13599135-13599253 : 3
	3:13599645-13599763 : 1
	3:13681862-13681980 : 5
	3:13682872-13682990 : 6
	3:13684418-13684536 : 1
	3:13689832-13689950 : 2
	3:13690318-13690436 : 1
	3:13694093-13694211 : 2
	3:13694599-13694717 : 1
	3:13696098-13696216 : 1
	3:13697681-13697799 : 1
	3:13698194-13698312 : 1
	3:13698699-13698817 : 1
	3:13699205-13699323 : 1
	3:13700728-13700846 : 2
	3:13702627-13702745 : 1
	3:13705427-13705545 : 1
	3:13706436-13706554 : 2
	3:13706941-13707059 : 5
	3:13707446-13707564 : 2
	3:13709971-13710089 : 1
	3:13710475-13710593 : 2
	3:1371

In [175]:
# test
from Bio import SeqIO
import gzip, time

def filterFastn(fastnin, reads_to_filter, fastnout, fastnformat="fastq", verbose=False,
                filterOut=False, LOG_EVERY_Nreads=1000000, compressed=True):
    
    """ filters a fast(a|q) file for read IDS in the list provided """
    
    print("Filtering fast(a|q) file %s to file %s..." % (fastnin, fastnout))
    
    reads_to_filter = set(reads_to_filter)
    if compressed:
        infile = gzip.open(fastnin, "rt")
        outfile = gzip.open(fastnout, "wt")
    else:
        infile = open(fastnin, "rt")
        outfile = open(fastnout, "wt")
    
    ngoodreads=0
    nlogs=1
    counter=0
    t0=time.time()
    ts=time.time()
    if filterOut:
        for record in SeqIO.parse(infile, fastnformat):
            if record.name not in reads_to_filter:
                c = SeqIO.write(record, outfile, fastnformat)
                ngoodreads+=1
            
            counter+=1
            if (counter % LOG_EVERY_Nreads)==0:
                thistime = time.time()-t0
                print("\tprocessed {} reads in {:.4f}s ({:.2f}reads/s)...".format((nlogs*LOG_EVERY_Nreads),
                                                                                        time.time()-ts,
                                                                                        (nlogs*LOG_EVERY_Nreads)/thistime))
                nlogs+=1
                t0=time.time()
    else:
        for record in SeqIO.parse(infile, fastnformat):
            if record.name not in reads_to_filter:
                c = SeqIO.write(record, outfile, fastnformat)
                ngoodreads+=1
            
            counter+=1
            if (counter % LOG_EVERY_Nreads)==0:
                thistime = time.time()-t0
                print("\tprocessed {} reads in {:.4f}s ({:.2f}reads/s)...".format((nlogs*LOG_EVERY_Nreads),
                                                                                        time.time()-ts,
                                                                                        (nlogs*LOG_EVERY_Nreads)/thistime))
                nlogs+=1
                t0=time.time()
    
    infile.close()
    outfile.close()
    return(fastnout)

In [176]:
fout = filterFastn("pathto/inputreads.fastq", filtered_out, "pathto/filtered_fastq.gz", filterOut=True)

Filtering fast(a|q) file /cluster/gsu/delivery/GSU_GS_RNAmethylation/Col-1/39_LIB1489_LDI1329_GCCAAT_L005_R1_001.fastq.gz to file /cluster/gjb_lab/nschurch/NOBACK/filtered_fastq.gz...
	processed 1000000 reads in 93.6436s (10678.79reads/s)...
	processed 2000000 reads in 93.9611s (21285.41reads/s)...
	processed 3000000 reads in 93.6889s (32020.86reads/s)...
	processed 4000000 reads in 93.1015s (42963.85reads/s)...
	processed 5000000 reads in 92.7558s (53905.01reads/s)...
	processed 6000000 reads in 92.5261s (64846.59reads/s)...
	processed 7000000 reads in 92.7822s (75445.48reads/s)...
	processed 8000000 reads in 92.3030s (86671.04reads/s)...
	processed 9000000 reads in 92.4541s (97345.58reads/s)...
	processed 10000000 reads in 91.5902s (109182.02reads/s)...
	processed 11000000 reads in 91.0740s (120780.94reads/s)...
	processed 12000000 reads in 90.3187s (132862.89reads/s)...
	processed 13000000 reads in 90.4333s (143752.34reads/s)...
	processed 14000000 reads in 89.4895s (156442.86reads/

In [168]:
rec.name

'HWI-ST1398:102:C42TMACXX:5:1101:6274:2000'

In [169]:
filtered_out

{'HWI-ST1398:102:C42TMACXX:5:2216:5346:59149',
 'HWI-ST1398:102:C42TMACXX:5:1313:1274:94282',
 'HWI-ST1398:102:C42TMACXX:5:1214:9444:26751',
 'HWI-ST1398:102:C42TMACXX:5:2114:12927:17694',
 'HWI-ST1398:102:C42TMACXX:5:1216:6449:86543',
 'HWI-ST1398:102:C42TMACXX:5:1114:10666:2157',
 'HWI-ST1398:102:C42TMACXX:5:1208:2878:35685',
 'HWI-ST1398:102:C42TMACXX:5:2110:14650:46128',
 'HWI-ST1398:102:C42TMACXX:5:2213:16463:74208',
 'HWI-ST1398:102:C42TMACXX:5:2311:17882:42876',
 'HWI-ST1398:102:C42TMACXX:5:2208:5442:32240',
 'HWI-ST1398:102:C42TMACXX:5:2116:11640:78153',
 'HWI-ST1398:102:C42TMACXX:5:2205:11306:67348',
 'HWI-ST1398:102:C42TMACXX:5:2112:17866:70238',
 'HWI-ST1398:102:C42TMACXX:5:2216:18549:17667',
 'HWI-ST1398:102:C42TMACXX:5:1310:9819:51171',
 'HWI-ST1398:102:C42TMACXX:5:2114:8211:69203',
 'HWI-ST1398:102:C42TMACXX:5:2112:4869:37227',
 'HWI-ST1398:102:C42TMACXX:5:2204:4281:14641',
 'HWI-ST1398:102:C42TMACXX:5:2102:7261:37963',
 'HWI-ST1398:102:C42TMACXX:5:2304:15361:61682',
 'HW